In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
#we should download the s&p file from some provider, but its paid
# so we will use an offline copy downloaded from somewhere
stocks=pd.read_csv('sp_500_stocks.csv')


In [3]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [4]:
from secrets import IEX_CLOUD_API_TOKEN

In [9]:
# price=data['latestPrice']
# cap=data['marketCap']

In [10]:
# print(price ,cap)

In [11]:
my_columns=['Ticker', 'StockPrice','MarketCap','NumStocksBuy']
# final_dataframe=pd.DataFrame(columns=my_columns)
# final_dataframe

In [12]:
# final_dataframe.append(
# pd.Series([
#     symbol,
#     price,
#     cap,
#     'N/A'
# ], index=my_columns),
#     ignore_index=True
# )

In [13]:
version='stable'# can be 'latest'
base_url=f'https://sandbox.iexapis.com/{version}'
# final_dataframe=pd.DataFrame(columns=my_columns)
# for stock in stocks['Ticker'][:5]:
#     api_url=f'{base_url}/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
#     data=requests.get(api_url).json()
#     final_dataframe=final_dataframe.append(
#     pd.Series(
#     [
#         stock,
#         data['latestPrice'],
#         data['marketCap'],
#         'N/A'
#     ],
#         index=my_columns
#     ),ignore_index=True
#     )

In [14]:
# final_dataframe

In [15]:
#for making batch api calls
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [16]:
symbol_groups=list(chunks(stocks['Ticker'],100))
# symbol_groups
symbol_strings=[]
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
# print (symbol_strings[0])

In [17]:
final_dataframe=pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url=f'{base_url}/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_call_url).json()
#     print(data.status_code)
    for symbol in symbol_string.split(","):
        final_dataframe=final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
        ], index=my_columns
        ),ignore_index=True
            
        )
final_dataframe  

,Ticker,StockPrice,MarketCap,NumStocksBuy
0,A,127.12,39377158109,N/A
1,AAL,19.20,12424333250,N/A
2,AAP,172.22,11748895987,N/A
3,AAPL,130.28,2218903940435,N/A
4,ABBV,110.10,192732287986,N/A
...,...,...,...,...
500,YUM,109.38,33150197119,N/A
501,ZBH,163.37,32665976792,N/A
502,ZBRA,493.69,26395609827,N/A
503,ZION,54.30,8496597526,N/A


In [27]:

def enterSize():
    portfolio_size=input('Enter the value of portfolio : ')
    try:
        psize=float(portfolio_size)
        
    except ValueError:
        print("Please enter a number \nPlease Try Again\n")
        psize=stringToFloat()
    return psize    
        


In [31]:
psize=enterSize()

Enter the value of portfolio : 1000000


In [32]:
print(psize)

1000000.0


In [33]:
position_size=psize/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'NumStocksBuy']=math.floor(position_size/final_dataframe.loc[i,'StockPrice'])
final_dataframe

,Ticker,StockPrice,MarketCap,NumStocksBuy
0,A,127.12,39377158109,15
1,AAL,19.20,12424333250,103
2,AAP,172.22,11748895987,11
3,AAPL,130.28,2218903940435,15
4,ABBV,110.10,192732287986,17
...,...,...,...,...
500,YUM,109.38,33150197119,18
501,ZBH,163.37,32665976792,12
502,ZBRA,493.69,26395609827,4
503,ZION,54.30,8496597526,36


In [53]:
writer=pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades',index=False)

In [54]:
background_color='#0a0a23'
font_color='#ffffff'
string_format=writer.book.add_format(
    {
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

dollar_format=writer.book.add_format(
    {
        'num_format':'$0.00',
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

integer_format=writer.book.add_format(
    {
        'num_format':'0',
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)


In [55]:
column_formats={
    'A':['Ticker',string_format],
    'B':['StockPrice',dollar_format],
    'C':['MarketCap',dollar_format],
    'D':['NumStocksBuy',integer_format]
}

In [56]:
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',22,column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1',column_formats[column][0],column_formats[column][1])
# writer.sheets['Recommended Trades'].write('A1','Ticker',string_format)
# writer.sheets['Recommended Trades'].write('B1','StockPrice',dollar_format)
# writer.sheets['Recommended Trades'].write('C1','MarketCap',dollar_format)
# writer.sheets['Recommended Trades'].write('D1','NumStocksBuy',integer_format)

In [57]:
writer.save()

In [51]:
print(final_dataframe)

    Ticker  StockPrice      MarketCap  NumStocksBuy
0        A      127.12    39377158109            15
1      AAL       19.20    12424333250           103
2      AAP      172.22    11748895987            11
3     AAPL      130.28  2218903940435            15
4     ABBV      110.10   192732287986            17
..     ...         ...            ...           ...
500    YUM      109.38    33150197119            18
501    ZBH      163.37    32665976792            12
502   ZBRA      493.69    26395609827             4
503   ZION       54.30     8496597526            36
504    ZTS      166.00    77507031700            11

[505 rows x 4 columns]
